<a href="https://colab.research.google.com/github/Freesoul-tech/Louis-Mahobe/blob/main/RNN35.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

class SimpleRNN:
    def __init__(self, n_features, n_nodes):
        self.n_features = n_features
        self.n_nodes = n_nodes

        self.w_x = np.random.randn(n_features, n_nodes) * 0.01
        self.w_h = np.random.randn(n_nodes, n_nodes) * 0.01
        self.b = np.ones(n_nodes)

    def forward(self, x):
        batch_size, n_sequences, _ = x.shape
        h = np.zeros((batch_size, self.n_nodes))

        self.h_list = []

        for t in range(n_sequences):
            x_t = x[:, t, :]
            a_t = np.dot(x_t, self.w_x) + np.dot(h, self.w_h) + self.b
            h = np.tanh(a_t)
            self.h_list.append(h)

        return h


In [2]:
x = np.array([[[1, 2], [2, 3], [3, 4]]]) / 100
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]]) / 100
w_h = np.array([[1, 3, 5, 7],
                [2, 4, 6, 8],
                [3, 5, 7, 8],
                [4, 6, 8, 10]]) / 100
b = np.array([1, 1, 1, 1])

# Manual override for testing
rnn = SimpleRNN(n_features=2, n_nodes=4)
rnn.w_x = w_x
rnn.w_h = w_h
rnn.b = b

output = rnn.forward(x)
print("Final hidden state:", output)


Final hidden state: [[0.79494228 0.81839002 0.83939649 0.85584174]]


In [3]:
class SimpleRNN:
    def __init__(self, n_features, n_nodes, learning_rate=0.01):
        self.n_features = n_features
        self.n_nodes = n_nodes
        self.learning_rate = learning_rate

        self.w_x = np.random.randn(n_features, n_nodes) * 0.01
        self.w_h = np.random.randn(n_nodes, n_nodes) * 0.01
        self.b = np.ones(n_nodes)

    def forward(self, x):
        batch_size, n_sequences, _ = x.shape
        h = np.zeros((batch_size, self.n_nodes))
        self.h_list = []
        self.a_list = []
        self.x_list = []

        for t in range(n_sequences):
            x_t = x[:, t, :]
            a_t = np.dot(x_t, self.w_x) + np.dot(h, self.w_h) + self.b
            h = np.tanh(a_t)

            self.h_list.append(h.copy())
            self.a_list.append(a_t.copy())
            self.x_list.append(x_t.copy())

        return h

    def backward(self, dL_dh_last):
        # Initialize gradients
        dW_x = np.zeros_like(self.w_x)
        dW_h = np.zeros_like(self.w_h)
        db = np.zeros_like(self.b)

        dL_dh_next = dL_dh_last

        for t in reversed(range(len(self.h_list))):
            h_t = self.h_list[t]
            a_t = self.a_list[t]
            x_t = self.x_list[t]
            h_prev = self.h_list[t - 1] if t > 0 else np.zeros_like(h_t)

            dL_da_t = dL_dh_next * (1 - np.tanh(a_t) ** 2)


            db += np.sum(dL_da_t, axis=0)
            dW_x += np.dot(x_t.T, dL_da_t)
            dW_h += np.dot(h_prev.T, dL_da_t)

            dL_dh_next = np.dot(dL_da_t, self.w_h.T)

        self.w_x -= self.learning_rate * dW_x
        self.w_h -= self.learning_rate * dW_h
        self.b -= self.learning_rate * db


In [4]:
import numpy as np

class ScratchSimpleRNNClassifier:
    def __init__(self, n_features, n_nodes, n_output, learning_rate=0.01):
        self.rnn = SimpleRNN(n_features, n_nodes, learning_rate)
        self.w_out = np.random.randn(n_nodes, n_output) * 0.01
        self.b_out = np.zeros(n_output)
        self.learning_rate = learning_rate

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def cross_entropy(self, y_pred, y_true):
        batch_size = y_pred.shape[0]
        return -np.sum(y_true * np.log(y_pred + 1e-7)) / batch_size

    def forward(self, x):
        h_last = self.rnn.forward(x)
        self.h_last = h_last
        self.z = np.dot(h_last, self.w_out) + self.b_out
        self.y_pred = self.softmax(self.z)
        return self.y_pred

    def backward(self, x, y_true):
        batch_size = y_true.shape[0]
        dL_dz = (self.y_pred - y_true) / batch_size

        dL_dw_out = np.dot(self.h_last.T, dL_dz)
        dL_db_out = np.sum(dL_dz, axis=0)


        dL_dh_last = np.dot(dL_dz, self.w_out.T)

        self.rnn.backward(dL_dh_last)


        self.w_out -= self.learning_rate * dL_dw_out
        self.b_out -= self.learning_rate * dL_db_out

    def train(self, x, y_true, epochs=10):
        for epoch in range(epochs):
            y_pred = self.forward(x)
            loss = self.cross_entropy(y_pred, y_true)
            self.backward(x, y_true)
            print(f"Epoch {epoch+1}, Loss: {loss:.4f}")


In [5]:

x = np.array([
    [[1, 2], [2, 3], [3, 4]],
    [[2, 1], [3, 2], [4, 3]]
]) / 100

y_true = np.array([
    [1, 0],
    [0, 1]
])

model = ScratchSimpleRNNClassifier(n_features=2, n_nodes=4, n_output=2, learning_rate=0.1)
model.train(x, y_true, epochs=20)


Epoch 1, Loss: 0.6932
Epoch 2, Loss: 0.6932
Epoch 3, Loss: 0.6932
Epoch 4, Loss: 0.6932
Epoch 5, Loss: 0.6932
Epoch 6, Loss: 0.6932
Epoch 7, Loss: 0.6932
Epoch 8, Loss: 0.6931
Epoch 9, Loss: 0.6931
Epoch 10, Loss: 0.6931
Epoch 11, Loss: 0.6931
Epoch 12, Loss: 0.6931
Epoch 13, Loss: 0.6931
Epoch 14, Loss: 0.6931
Epoch 15, Loss: 0.6931
Epoch 16, Loss: 0.6931
Epoch 17, Loss: 0.6931
Epoch 18, Loss: 0.6931
Epoch 19, Loss: 0.6931
Epoch 20, Loss: 0.6931
